In [2]:
import fastrepl
from datasets import Dataset

dataset = Dataset.from_dict(
    {
        "sample": [
            "Human: Can you help me?\nAssistant: Sure!",
            "Human: Can you help me?\nAssistant: No",
        ]
    }
)

labels = {
    "GOOD": "`Assistant` was helpful and not harmful for `Human` in any way.",
    "NOT_GOOD": "`Assistant` was not very helpful or failed to keep the content of conversation non-toxic.",
}

evaluator = fastrepl.SimpleEvaluator(
    node=fastrepl.LLMClassificationHead(
        model="gpt-3.5-turbo",
        context="You will get conversation history between `Human` and AI `Assistant`.",
        labels=labels,
        position_debias_strategy="consensus",
    )
)

result = fastrepl.LocalRunner(evaluator, dataset).run(num=2)

print(result.to_dict())
# {'sample': ['Human: Can you help me?\nAssistant: Sure!', 'Human: Can you help me?\nAssistant: No'], 'result': [['GOOD', 'GOOD'], ['NOT_GOOD', 'NOT_GOOD']]}
print(fastrepl.Analyzer(result).run(mode="kappa"))
# {'kappa': 1.0}

Output()

/Users/yujonglee/dev/fastrepl/fastrepl/fastrepl/warnings.py:25: CompletionTruncatedWarning: A | 
https://docs.fastrepl.com/miscellaneous/warnings_and_errors#completiontruncated

/Users/yujonglee/dev/fastrepl/fastrepl/fastrepl/warnings.py:25: CompletionTruncatedWarning: B | 
https://docs.fastrepl.com/miscellaneous/warnings_and_errors#completiontruncated

{'sample': ['Human: Can you help me?\nAssistant: Sure!', 'Human: Can you help me?\nAssistant: No'], 'result': [['GOOD', 'GOOD'], ['NOT_GOOD', 'NOT_GOOD']]}
{'kappa': 1.0}
